In [1]:
import os
from re import compile
from loaders.s3_bucket import S3Bucket
from pyspark.sql import DataFrame, SparkSession, Row
from pyspark.sql.functions import (
    monotonically_increasing_id,
    regexp_extract,
    when,
    col,
    lit,
    regexp_replace,
    first,
    coalesce,
    sum,
    explode,
    udf,
)
from pyspark.sql.window import Window
from pyspark.sql.types import (
    StructType,
    StructField,
    StringType,
    IntegerType,
    ArrayType,
)

development_phase_flag = "development"

In [2]:
hadoop_aws_version = "3.3.4"
aws_java_sdk_version = "1.12.319"

builder = (
    SparkSession.builder.appName("LoadGamesFromS3")
    .config(
        "spark.jars.packages",
        f"org.apache.hadoop:hadoop-aws:{hadoop_aws_version},com.amazonaws:aws-java-sdk-bundle:{aws_java_sdk_version}",
    )
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config(
        "spark.hadoop.fs.s3a.aws.credentials.provider",
        "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider",
    )
    .config("spark.hadoop.fs.s3a.access.key", os.environ["S3_ACCESS_KEY_ID"])
    .config("spark.hadoop.fs.s3a.secret.key", os.environ["S3_SECRET_ACCESS_KEY"])
    .config("spark.hadoop.fs.s3a.endpoint", "https://s3.cubbit.eu")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "true")
)
spark: SparkSession = builder.getOrCreate()

24/10/02 23:26:43 WARN Utils: Your hostname, Joses-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 172.20.10.2 instead (on interface en0)
24/10/02 23:26:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/jcbraz/Projects/chess-pipeline/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/jcbraz/.ivy2/cache
The jars for the packages stored in: /Users/jcbraz/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-705e97f7-3d58-4d6a-87eb-cc5e7f8755af;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.319 in central
:: resolution report :: resolve 479ms :: artifacts dl 17ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.319 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	:: evicted modules:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 by [com.amazonaws#aws-java-sdk-bundle;1.12.319] in [default]
	-------------------------------------------------------------

In [3]:
s3_bucket = S3Bucket(
    spark=spark,
    object_key="lichess_standard_rated_2024-08.pgn.zst",
    bucket_name="chess-pipeline",
    endpoint="https://s3.cubbit.eu",
    region="eu-west-1",
    s3_access_key=os.environ["S3_ACCESS_KEY_ID"],
    s3_secret_key=os.environ["S3_SECRET_ACCESS_KEY"],
)

In [4]:
# read csv from data dir
if development_phase_flag == "development":
    games_df: DataFrame = (
        spark.read.csv("../data/sample_games.csv", header=True)
        .filter(
            (col("value") != "")
            & (~col("value").like("%UTCTime%"))
            & (~col("value").like("%Result%"))
        )
        .withColumn("value", regexp_replace("value", '"', ""))
        .drop("_c0")
    )

    games_df.show()
else:

    games_df = (
        s3_bucket.load_games_dataframe_from_s3()
        .filter(
            (col("value") != "")
            & (~col("value").like("%UTCTime%"))
            & (~col("value").like("%Result%"))
        )
        .withColumn("value", regexp_replace("value", '"', ""))
        .drop("_c0")
    )

+--------------------+
|               value|
+--------------------+
|[Event Rated Bull...|
|[Site https://lic...|
|   [Date 2024.08.01]|
|           [Round -]|
|[White kingskreamer]|
| [Black mysteryvabs]|
|[UTCDate 2024.08.01]|
|     [WhiteElo 2148]|
|     [BlackElo 2155]|
|[WhiteRatingDiff +6]|
|[BlackRatingDiff -6]|
|           [ECO B10]|
|[Opening Caro-Kan...|
|  [TimeControl 60+0]|
|[Termination Time...|
|1. e4 { [%clk 0:0...|
|[Event Rated Bull...|
|[Site https://lic...|
|   [Date 2024.08.01]|
|           [Round -]|
+--------------------+
only showing top 20 rows



In [5]:
# Rename the "value" column to "Line"
games_df = games_df.withColumnRenamed("value", "Line")


# Extract the "Key" and "Value" columns based on the structure of the "Line" column
games_df = (
    games_df.withColumn(
        "Key",
        when(col("Line").startswith("1."), "Moves").otherwise(
            regexp_extract(col("Line"), r"\[(.*?)\s", 1)
        ),
    )
    .withColumn(
        "Value",
        when(col("Line").startswith("1."), col("Line")).otherwise(
            regexp_replace(col("Line"), r"\[\w+\s", ""),
        ),
    )
    .withColumn("Value", regexp_replace(col("Value"), r"\]", ""))
)

# Add a column to identify the start of a game
games_df = games_df.withColumn(
    "StartOfGame", when(col("Line").startswith("[Event"), 1).otherwise(lit(0))
)

# Define a window specification for calculating the cumulative sum
windowSpec = Window.orderBy(monotonically_increasing_id())

# Calculate the cumulative sum of "StartOfGame" to create "GameID"
games_df = games_df.withColumn("GameID", sum(col("StartOfGame")).over(windowSpec))

games_df.show()

24/10/02 23:26:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/02 23:26:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/02 23:26:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/02 23:27:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/02 23:27:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------------------+---------------+--------------------+-----------+------+
|                Line|            Key|               Value|StartOfGame|GameID|
+--------------------+---------------+--------------------+-----------+------+
|[Event Rated Bull...|          Event|   Rated Bullet game|          1|     1|
|[Site https://lic...|           Site|https://lichess.o...|          0|     1|
|   [Date 2024.08.01]|           Date|          2024.08.01|          0|     1|
|           [Round -]|          Round|                   -|          0|     1|
|[White kingskreamer]|          White|        kingskreamer|          0|     1|
| [Black mysteryvabs]|          Black|         mysteryvabs|          0|     1|
|[UTCDate 2024.08.01]|        UTCDate|          2024.08.01|          0|     1|
|     [WhiteElo 2148]|       WhiteElo|                2148|          0|     1|
|     [BlackElo 2155]|       BlackElo|                2155|          0|     1|
|[WhiteRatingDiff +6]|WhiteRatingDiff|              

In [6]:
col_list = (
    games_df.select("Key")
    .groupBy("Key").count()
    .filter(col("Key") != "")
    .toPandas()["Key"]
    .tolist()
)

col_list

['TimeControl',
 'Round',
 'ECO',
 'Event',
 'WhiteElo',
 'WhiteTitle',
 'BlackElo',
 'Termination',
 'BlackRatingDiff',
 'Opening',
 'Moves',
 'WhiteRatingDiff',
 'White',
 'BlackTitle',
 'UTCDate',
 'Date',
 'Black',
 'Site']

In [7]:
# Pivot the DataFrame based on "GameID" and the specified columns
pivot_games_df = (
    games_df.groupBy("GameID")
    .pivot("Key", col_list)
    .agg(first("Value"))
    .orderBy("GameID")
)

pivot_games_df = pivot_games_df.filter(col("Moves").contains("%eval"))
pivot_games_df.show()

24/10/02 23:27:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/02 23:27:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/02 23:27:03 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/10/02 23:27:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/02 23:27:04 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+------+-----------+-----+---+--------------------+--------+----------+--------+------------+---------------+--------------------+--------------------+---------------+-------------------+----------+----------+----------+-------------+--------------------+
|GameID|TimeControl|Round|ECO|               Event|WhiteElo|WhiteTitle|BlackElo| Termination|BlackRatingDiff|             Opening|               Moves|WhiteRatingDiff|              White|BlackTitle|   UTCDate|      Date|        Black|                Site|
+------+-----------+-----+---+--------------------+--------+----------+--------+------------+---------------+--------------------+--------------------+---------------+-------------------+----------+----------+----------+-------------+--------------------+
|    40|      180+2|    -|B50|Rated Blitz tourn...|    1845|      NULL|    1888|      Normal|             -7|Sicilian Defense:...|1. e4 { [%eval 0....|             +7|      TenderBeastXL|      NULL|2024.08.01|2024.08.01|    G_Capell

In [8]:
# Extract the "Round" column from the "Moves" column
pivot_games_df.select("Round").distinct().show(truncate=False)
pivot_games_df = pivot_games_df.drop("Round")

24/10/02 23:27:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/02 23:27:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/02 23:27:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/02 23:27:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----+
|Round|
+-----+
|-    |
+-----+



In [9]:
# Select the "GameID" and "Moves" columns
id_and_moves_df = pivot_games_df.select(["GameID", "Moves"]).repartition("GameID")
id_and_moves_df.show()

24/10/02 23:27:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/02 23:27:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/02 23:27:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/02 23:27:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+------+--------------------+
|GameID|               Moves|
+------+--------------------+
|    65|1. Nf3 { [%eval 0...|
|   367|1. d4 { [%eval 0....|
|   130|1. e4 { [%eval 0....|
|   262|1. e4 { [%eval 0....|
|   325|1. d4 { [%eval 0....|
|    50|1. e4 { [%eval 0....|
|   110|1. d4 { [%eval 0....|
|   190|1. e4 { [%eval 0....|
|   275|1. e4 { [%eval 0....|
|   395|1. e4 { [%eval 0....|
|    71|1. d4 { [%eval 0....|
|    58|1. e4 { [%eval 0....|
|   447|1. b3 { [%eval 0....|
|    52|1. e4 { [%eval 0....|
|   209|1. e4 { [%eval 0....|
|   208|1. e4 { [%eval 0....|
|    89|1. e4 { [%eval 0....|
|   411|1. e4 { [%eval 0....|
|   142|1. f4 { [%eval -0...|
|   132|1. e4 { [%eval 0....|
+------+--------------------+
only showing top 20 rows



In [10]:
# Define a UDF to parse the game moves and evaluations
re_pattern = compile(
    r"'?(\d+)\.(\w?\-?\w+\!*?\?*?\!?)!?\??\{\[%eval(-?\d+\.\d+)\[%clk(\d+:\d+:\d+)\}(\d+)\.{3}(\w?\-?\w+\!*?\?*?\!?)\{\[%eval(-?\d+\.\d+)\[%clk(\d+:\d+:\d+)\}(\d-\d)?'?"
)


def parse_game_moves(game_string: str) -> list[Row]:
    """
    Parse the game string to extract the moves and evaluations.

    Args:
    game_string: str
        The string containing the game moves.

    Returns:
        list[Row]: The list of parsed moves and evaluations.
    """
    no_spacing_games_string = game_string.replace(" ", "")
    try:
        moves: list[str] = re_pattern.findall(no_spacing_games_string)
    except Exception as e:
        print(f"Error applying regex into moves: {e}")
        return []

    return [
        Row(
            WhiteMoveCount=int(move[0]) if move[0] else None,
            WhiteMove=move[1] if move[1] else None,
            WhiteEval=move[2] if move[2] else None,
            WhiteTime=move[3] if move[3] else None,
            BlackMoveCount=int(move[4]) if move[4] else None,
            BlackMove=move[5] if move[5] else None,
            BlackEval=move[6] if move[6] else None,
            BlackTime=move[7] if move[7] else None,
            Result=move[8] if move[8] else None,
        )
        for move in moves
    ]

In [11]:
# Define the schema for the parsed moves
schema = StructType(
    [
        StructField("WhiteMoveCount", IntegerType(), True),
        StructField("WhiteMove", StringType(), True),
        StructField("WhiteEval", StringType(), True),
        StructField("WhiteTime", StringType(), True),
        StructField("BlackMoveCount", IntegerType(), True),
        StructField("BlackMove", StringType(), True),
        StructField("BlackEval", StringType(), True),
        StructField("BlackTime", StringType(), True),
        StructField("Result", StringType(), True),
    ]
)

# UDF to parse game moves
parse_game_moves_udf = udf(parse_game_moves, ArrayType(schema))

# Apply UDF and explode the resulting array of structs
parsed_moves_df = id_and_moves_df.withColumn(
    "ParsedMoves", explode(parse_game_moves_udf(col("Moves")))
)

# Select the necessary columns and add GameID
normalized_moves_df = parsed_moves_df.select(
    col("GameID"),
    col("ParsedMoves.WhiteMoveCount"),
    col("ParsedMoves.WhiteMove"),
    col("ParsedMoves.WhiteEval"),
    col("ParsedMoves.WhiteTime"),
    col("ParsedMoves.BlackMoveCount"),
    col("ParsedMoves.BlackMove"),
    col("ParsedMoves.BlackEval"),
    col("ParsedMoves.BlackTime"),
    col("ParsedMoves.Result"),
)

normalized_moves_df.show()

24/10/02 23:27:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/02 23:27:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/02 23:27:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/02 23:27:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+------+--------------+---------+---------+---------+--------------+---------+---------+---------+------+
|GameID|WhiteMoveCount|WhiteMove|WhiteEval|WhiteTime|BlackMoveCount|BlackMove|BlackEval|BlackTime|Result|
+------+--------------+---------+---------+---------+--------------+---------+---------+---------+------+
|    65|             1|      Nf3|     0.17|  0:03:00|             1|      Nf6|     0.22|  0:03:00|  NULL|
|    65|             2|       g3|     0.11|  0:03:01|             2|       g6|     0.17|  0:03:02|  NULL|
|    65|             3|       b3|    -0.01|  0:03:02|             3|      Bg7|     0.19|  0:03:04|  NULL|
|    65|             4|      Bb2|     0.09|  0:03:03|             4|       d6|     0.14|  0:03:05|  NULL|
|    65|             5|       d4|     0.08|  0:03:04|             5|      O-O|     0.21|  0:03:05|  NULL|
|    65|             6|      Bg2|      0.2|  0:03:02|             6|      Bg4|     0.45|  0:03:05|  NULL|
|    65|             7|     Nbd2|     0.33|  0

In [12]:
moves_values = id_and_moves_df.select("Moves").rdd.flatMap(lambda x: x).collect()
parsed_moves_rows = [parse_game_moves(move) for move in moves_values]
normalized_moves_df = spark.createDataFrame(
    data=[move for moves in parsed_moves_rows for move in moves],
    schema="WhiteMoveCount INT, WhiteMove STRING, WhiteEval STRING, WhiteTime STRING, BlackMoveCount INT, BlackMove STRING, BlackEval STRING, BlackTime STRING, Result STRING",
)

# temporary variable
normalized_moves_df = normalized_moves_df.withColumn(
    "row_num", monotonically_increasing_id()
)

# temporary variable
game_id_df = id_and_moves_df.select("GameID").withColumn(
    "row_num", monotonically_increasing_id()
)

# Join the DataFrames on the "row_num" column and drop the "row_num" column
normalized_moves_df = normalized_moves_df.join(game_id_df, "row_num", "left").drop(
    "row_num"
)

normalized_moves_df.show()

24/10/02 23:27:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/02 23:27:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/02 23:27:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/02 23:27:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/02 23:27:14 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/02 23:27:14 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/02 2

+--------------+---------+---------+---------+--------------+---------+---------+---------+------+------+
|WhiteMoveCount|WhiteMove|WhiteEval|WhiteTime|BlackMoveCount|BlackMove|BlackEval|BlackTime|Result|GameID|
+--------------+---------+---------+---------+--------------+---------+---------+---------+------+------+
|             1|      Nf3|     0.17|  0:03:00|             1|      Nf6|     0.22|  0:03:00|  NULL|    65|
|             2|       g3|     0.11|  0:03:01|             2|       g6|     0.17|  0:03:02|  NULL|   367|
|             3|       b3|    -0.01|  0:03:02|             3|      Bg7|     0.19|  0:03:04|  NULL|   130|
|             4|      Bb2|     0.09|  0:03:03|             4|       d6|     0.14|  0:03:05|  NULL|   262|
|             5|       d4|     0.08|  0:03:04|             5|      O-O|     0.21|  0:03:05|  NULL|   325|
|             6|      Bg2|      0.2|  0:03:02|             6|      Bg4|     0.45|  0:03:05|  NULL|    50|
|             7|     Nbd2|     0.33|  0:03:00|

In [13]:
normalized_moves_df = normalized_moves_df.withColumn(
    "Result",
    when(
        (col("Result").isNull())
        & (col("WhiteMove").contains("-"))
        & (col("BlackMove").rlike(r"-")),
        coalesce(col("WhiteMove"), col("BlackMove")),
    ).otherwise(col("Result")),
)

normalized_moves_df.show()

24/10/02 23:27:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/02 23:27:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/02 23:27:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/02 23:27:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------------+---------+---------+---------+--------------+---------+---------+---------+------+------+
|WhiteMoveCount|WhiteMove|WhiteEval|WhiteTime|BlackMoveCount|BlackMove|BlackEval|BlackTime|Result|GameID|
+--------------+---------+---------+---------+--------------+---------+---------+---------+------+------+
|             1|      Nf3|     0.17|  0:03:00|             1|      Nf6|     0.22|  0:03:00|  NULL|    65|
|             2|       g3|     0.11|  0:03:01|             2|       g6|     0.17|  0:03:02|  NULL|   367|
|             3|       b3|    -0.01|  0:03:02|             3|      Bg7|     0.19|  0:03:04|  NULL|   130|
|             4|      Bb2|     0.09|  0:03:03|             4|       d6|     0.14|  0:03:05|  NULL|   262|
|             5|       d4|     0.08|  0:03:04|             5|      O-O|     0.21|  0:03:05|  NULL|   325|
|             6|      Bg2|      0.2|  0:03:02|             6|      Bg4|     0.45|  0:03:05|  NULL|    50|
|             7|     Nbd2|     0.33|  0:03:00|

## Ingestion back into S3 

In [14]:
try:
    s3_bucket.load_games_to_s3(normalized_moves_df, "moves_df")
except Exception as e:
    print(e)

24/10/02 23:27:19 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
24/10/02 23:27:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/02 23:27:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/02 23:27:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/02 23:27:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/02 23:27:31 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [15]:
filtered_pivot_games_df = pivot_games_df.drop("Moves")

In [16]:
try:
    s3_bucket.load_games_to_s3(filtered_pivot_games_df, "games_df")
except Exception as e:
    print(e)

24/10/02 23:28:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/02 23:28:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/02 23:28:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/10/02 23:28:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [17]:
spark.stop()